In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow_probability.substrates import numpy as tfp

In [ ]:
!pip install cmdstanpy
!pip install git+https://github.com/OriolAbril/arviz.git@ci

In [ ]:
from cmdstanpy import install_cmdstan
install_cmdstan()

In [2]:
from cmdstanpy import CmdStanModel, set_cmdstan_path
import arviz as az

In [76]:
X = pd.read_csv("../Datasets/covariates.csv")

In [61]:
X.head()
Y.head()

,Count_120,idSensore,Year,Month
0,4.0,5707,2010,4
1,8.0,5707,2010,5
2,16.0,5707,2010,6
3,25.0,5707,2010,7
4,12.0,5707,2010,8


In [78]:
X['Type'] = X['Type'].astype('category')

In [79]:
X = X.drop('Type', axis = 1)

In [77]:
Y = pd.read_csv("../Datasets/Dataset_120.csv").sort_values(by=['idSensore','Year','Month'])
Y.head()
Y.shape

(4095, 4)

In [81]:
rows_to_drop_Y = Y[Y['Count_120'].isna()].index

# Remove corresponding rows from dataset X
X = X.drop(index=rows_to_drop_Y)
Y = Y.drop(index=rows_to_drop_Y)
stations = X['Station']
years = X['Year']

# If you want to reset the index after dropping rows
X = X.reset_index(drop=True)

X.shape


(3840, 12)

In [25]:
prior_elic = """

data {
  int<lower=1> N; // Number of observations
  int<lower=1> P; // Covariate number (should be around 10)
  int<lower=1> nyears;
  int<lower=1> nstations;


  array[N] int<lower=1> station;
  array[N] int<lower=2010, upper=2022> year;
  // array[N] int<lower=4, upper=10> month;

  array[N] int<lower=0> y; // Count data
  matrix[N, P] X; // Predictor matrix
}

parameters {
  vector[P] beta; // Coefficients for predictors
  vector[nstations] eta; // Random effects for comuni
  vector[nyears] xi; // Random effects for years

  //real<lower=0> sigma0; // Standard deviation for beta
  //real<lower=0> sigma1; // Standard deviation for xi
  //real<lower=0> sigma2; // Standard deviation for eta
}

transformed parameters {
    vector[N] lambda;
    vector[N] intercept;
    vector[N] fix_eff;

    intercept = xi[year] + eta[station];
    fix_eff = X * beta;

    lambda = exp(intercept + fix_eff);
}

model {

  beta ~ normal(0, 10);
  xi ~ normal(0, 10);
  eta ~ normal(0, 10);

  for (n in 1:N) {
    y[n] ~ poisson(lambda[n]);
  }

}

"""

stan_file = "./priors.stan"

with open(stan_file, "w") as f:
    print(prior_elic, file=f)

priors = CmdStanModel(stan_file=stan_file)

18:08:02 - cmdstanpy - INFO - compiling stan file C:\Users\leoma\OneDrive\DOCUME~1\PoliMi\BAYESI~1\BAYESI~1\main\Stan\priors.stan to exe file C:\Users\leoma\OneDrive\Documents\PoliMi\Bayesian statistics\Bayesian_Project\main\Stan\priors.exe
18:08:36 - cmdstanpy - INFO - compiled model executable: C:\Users\leoma\OneDrive\Documents\PoliMi\Bayesian statistics\Bayesian_Project\main\Stan\priors.exe


In [80]:
Y['Count_120'] = pd.to_numeric(Y['Count_120']).astype('Int64')

In [83]:
data = {
    "N": len(Y),
    "P": X.shape[1],
    "nyears": 13,
    "nstations": 45,
    "station": stations,
    "year": years,
    "y": Y['Count_120'],
    "X": X
}


fit = priors.sample(data=data, chains=4, parallel_chains=4, 
    iter_warmup=1000, iter_sampling=1000)
prior_az = az.from_cmdstanpy(fit)

18:26:41 - cmdstanpy - INFO - CmdStan start processing


chain 1 |          | 00:00 Status

chain 2 |          | 00:00 Status

chain 3 |          | 00:00 Status

chain 4 |          | 00:00 Status

18:26:41 - cmdstanpy - ERROR - Chain [2] error: error during processing Operation not permitted
18:26:41 - cmdstanpy - ERROR - Chain [3] error: error during processing Operation not permitted
18:26:41 - cmdstanpy - ERROR - Chain [1] error: error during processing Operation not permitted
18:26:41 - cmdstanpy - ERROR - Chain [4] error: error during processing Operation not permitted


18:26:41 - cmdstanpy - INFO - CmdStan done processing.


RuntimeError: Error during sampling:
Exception: vector[multi] indexing: accessing element out of range. index 5707 out of range; expecting index to be between 1 and 45 (in 'priors.stan', line 33, column 4 to column 40)
Exception: vector[multi] indexing: accessing element out of range. index 5707 out of range; expecting index to be between 1 and 45 (in 'priors.stan', line 33, column 4 to column 40)
Exception: vector[multi] indexing: accessing element out of range. index 5707 out of range; expecting index to be between 1 and 45 (in 'priors.stan', line 33, column 4 to column 40)
Exception: vector[multi] indexing: accessing element out of range. index 5707 out of range; expecting index to be between 1 and 45 (in 'priors.stan', line 33, column 4 to column 40)
Command and output files:
RunSet: chains=4, chain_ids=[1, 2, 3, 4], num_processes=4
 cmd (chain 1):
	['C:\\Users\\leoma\\OneDrive\\Documents\\PoliMi\\Bayesian statistics\\Bayesian_Project\\main\\Stan\\priors.exe', 'id=1', 'random', 'se